<a href="https://colab.research.google.com/github/ojw92/NLP-for-Text-Classification/blob/main/BERT_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
#@title Preprocessing (not needed for BERT)

# preprocessor.py

# Preprocessing steps for RNN & CNN

# Model for S22 VOC only, but with added preprocessing steps

# https://www.geeksforgeeks.org/python-call-function-from-another-file/

from timeit import default_timer as timer

import pandas as pd
import numpy as np
import re
import nltk
nltk.download('wordnet')    # for lemmatization function; don't need this line if package installed
nltk.download('omw-1.4')    # for lemmatization function; don't need this line if package installed
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer     # based on The Porter Stemming Algorithm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datetime import datetime, timedelta
import os
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91

# 0 - split & prepare data for preprocessing                              <- add a preliminary stopword removal step (RNN & CNN)?
# 0.5 - remove duplicate data
# 1 - remove hyperlinks, emails & pricetags
# 2 - remove emoji
# 3 - remove punctuations
# 4 - apply lower case

# ===== use min-df to find low-frequency words =====

# 5 - fix misspellings                                                    <- (need to implement!)
# 6 - replace words with generic nouns                                    <- (need to implement!)
# 7 - remove stopwords       # better to replace then remove?             <- final stopword clean-up step (separate list for RNN & CNN)
# 8 - lemmatize
# (8.5 - stemming)

# 9 - tokenization & padding

# ============================================================================================================= 0                 <- add a preliminary stopword removal step

def data_split(df22, R_known=True):

    # Split the input dataframe into Text (Title + Contents) and Classes dataframes
    # input must have 3 columns of string entries (Title, Contents, and Classes)

    
    # print(df22.describe())     # get an overview of the dataset

    # check if every row entry of each column is string type (some are NaN, so no)
    # print(df22.applymap(lambda x : type(x).__name__).eq({'Title': 'str', 'Content': 'str', 'Class':'str'}))

    # convert NaN to empty strings (NaN -> str)
        # df22.apply(str) converts all columns to str, as well
    df22 = df22.replace(float('nan'), '', regex=True)

    # concatenate strings of title & content with a " " in between (1 body of text)
    text22 = df22['Title'] + " " + df22['Content']      # slicing DataFrame via .iloc[:,0] makes it a Series
    text22 = pd.DataFrame(text22, columns= ['Text'])    # so initialize it as a DataFrame. pd.DataFrame(some_Series) works
    classes22 = df22['Class']

    if R_known == True:
    # R, r, YR = 1;     N, n, YN = 0
        R_cases = re.compile('R|YR', re.IGNORECASE)
        N_cases = re.compile('N|YN', re.IGNORECASE)
        classes22 = classes22.replace(to_replace=R_cases, value=1)
        classes22 = classes22.replace(to_replace=N_cases, value=0)
    else:
        # R_known == False; prepping not yet classified data
        Y_N_cases = re.compile('Y|N', re.IGNORECASE)
        classes22 = classes22.replace(to_replace=Y_N_cases, value=0)     # all N's for simplicity

    classes22 = pd.DataFrame(classes22, columns=['Class']).astype('int32')
    # classes22.columns = ['Class']    # classes22=pd.DataFrame(classes22) causes an error cus   pd.DataFrame(some_DataFrame) makes no sense - should pass a list

    #print(text22.head(10))
    #print(classes22.head(10))
    print('==================================================')

    return text22, classes22



# ============================================================================================================= 0.5

def drop_dupe_text(text22, classes22):

    # df22.drop_duplicates() will remove extra rows that have the same values in all columns (redundant)
    # As a result only duplicate rows left are pairs of datapoints with 0 & 1 as their labels
    # This function removes all of rows from those pairs with 0 as their label, as they can be disruptive
    # in training the model. Label 1 should take priority

    textdupeindex = text22[text22.duplicated(keep=False)==True].index
    dupeclasses = classes22.filter(items = textdupeindex, axis=0)
    redundantclass0 = dupeclasses[dupeclasses.Class==0]
    text22.drop(index=redundantclass0.index, inplace=True)
    classes22.drop(index=redundantclass0.index, inplace=True)

    return text22, classes22





[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### BERT Practice
#### v. 20230118

In [57]:
# pip3 install torch torchvision torchaudio     # for PyTorch without GPU, just CPU


In [58]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
# Check if the GPU can be detected
import tensorflow as tf

device_name = tf.test.gpu_device_name()              # '/device:GPU:0' means GPU is enabled
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [60]:
import torch
# torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [61]:
!cat /proc/meminfo            # check memory resources available

MemTotal:       13297192 kB
MemFree:          565624 kB
MemAvailable:    7973144 kB
Buffers:          362748 kB
Cached:          7137612 kB
SwapCached:            0 kB
Active:          1187008 kB
Inactive:       11033440 kB
Active(anon):        916 kB
Inactive(anon):  4708412 kB
Active(file):    1186092 kB
Inactive(file):  6325028 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1552 kB
Writeback:             0 kB
AnonPages:       4720160 kB
Mapped:          1103328 kB
Shmem:             15584 kB
KReclaimable:     222452 kB
Slab:             277096 kB
SReclaimable:     222452 kB
SUnreclaim:        54644 kB
KernelStack:        5184 kB
PageTables:        41860 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648596 kB
Committed_AS:    6739124 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       53756 kB
VmallocChunk:          0 kB
Percpu:          

In [62]:
# install wandb for tracking data on dashboard
!pip install datasets wandb evaluate -qU
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

--2023-01-19 21:22:55--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27264 (27K) [text/plain]
Saving to: ‘run_glue.py.1’

run_glue.py.1       100%[===================>]  26.62K  --.-KB/s    in 0s      

2023-01-19 21:22:55 (77.9 MB/s) - ‘run_glue.py.1’ saved [27264/27264]



In [63]:
# the run_glue.py script requires transformers dev
!pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [64]:
import wandb

# log in to have data synced to account
# wandb.login()

# log every trained model
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [65]:

# https://github.com/PradipNichite/Youtube-Tutorials/blob/main/FineTune_BERT_Model_Youtube.ipynb



import pandas as pd

df22 = pd.read_csv('/S22_train20221011.csv', sep='\t', index_col=0).drop_duplicates()

# repair rows where values are shifted by 1 column
shift_rows = df22.loc[df22[df22.Class.apply(lambda x: x == 'None')].index]   # rows where columns shifted to left
shift_values = shift_rows[df22.columns[1:-1]]
shift_rows[df22.columns[2:]] = shift_rows[df22.columns[1:-1]]
shift_rows.Content = ''
df22.loc[df22[df22.Class.apply(lambda x: x == 'None')].index] = shift_rows

# filter training data to desired dates
df22 = df22[df22.Date.apply(lambda x: datetime.strptime(x,'%m/%d/%Y %H:%M').date() > datetime(2022,2,21).date())]

# For class imbalance, use roughly same ratio of R & N
######################################################### BERT doesn't need class imbalance addressed
# df22 = pd.concat([df22[df22.Class=='R'], df22[df22.Class=='N'].iloc[::3, :]])

# combine title & content as text22, clean the text, then combine it with labels to a single df  
text22, classes22 = data_split(df22)
df22, classes22 = drop_dupe_text(text22, classes22)

# Preprocessing & data cleaning not required; BERT uses all info in sentence (punctuation, stopwords, etc)   
df22['Class'] = classes22['Class']

df22.head()




################################ DELETE THIS LINE LATER (purpose: reduce data to reduce training time to test features)
# df22 = df22.sample(frac=0.01, random_state=5)




,Text,Class
452,Zfold 3 or S22 ultra Trying to decide between ...,0
453,S22 video cam Anyone tried out the video camer...,0
454,Thinking about trading my S21Ultra for S22+ Ev...,0
455,S21 Ultra vs S22+ Both phones are currently at...,0
456,"S21 or S22 Base/Standard Model Hey All,\n\nLoo...",0


In [66]:
df22['Class'].value_counts()

0    13926
1     3658
Name: Class, dtype: int64

In [67]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer


from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_positio

In [68]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [69]:
# model = model.to( torch.device('cuda') )     # need NVIDIA driver for 'cuda'; currently have AMD on work laptop
# model = model.to('cpu')         # train on CPU

model = model.to('cuda')          # or  model.cuda()


In [70]:

test_data = ["This is possibly the worst battery I have ever seen on a mobile device",
            "How is my device running so smoothly?"]
tokenizer(test_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 2023, 2003, 4298, 1996, 5409, 6046, 1045, 2031, 2412, 2464, 2006, 1037, 4684, 5080, 102], [101, 2129, 2003, 2026, 5080, 2770, 2061, 15299, 1029, 102, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [71]:
X = list(df22["Text"])
y = list(df22["Class"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [72]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [73]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [74]:
len(X_train),len(X_val)


(14067, 3517)

In [75]:
# Create torch dataset
class VOC_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [76]:
train_dataset = VOC_Dataset(X_train_tokenized, y_train)
val_dataset = VOC_Dataset(X_val_tokenized, y_val)

In [77]:
train_dataset[5]


{'input_ids': tensor([  101,  1055, 19317, 11087,  2417,  7262,  3475,  1005,  1056,  1012,
          1012,  1012,  2417,  1029,  2633,  2288,  2026,  1055, 19317, 11087,
          5359,  2651,  1998,  2009,  1005,  1055,  2025,  2130,  2485,  2000,
          2417,  1012,  2009,  1005,  1055, 11034,  2012,  2190,  1012,  2428,
          1010,  2428,  9364,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [78]:
def compute_metrics(m):
    print(type(m))
    pred, labels = m
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
     

In [79]:
# Define Trainer
args = TrainingArguments(
    report_to = 'wandb',                     # enable logging to W&B
    output_dir="output",                     # output directory
    num_train_epochs=1,
    per_device_train_batch_size=8
    # overwrite_output_dir = True,
    # evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    # learning_rate = 5e-5,                   # we can customize learning rate
    # max_steps = 30000,
    # logging_steps = 100,                    # we will log every 100 steps
    # eval_steps = 5000,                      # we will perform evaluation every 500 steps
    # save_steps = 10000,
    # load_best_model_at_end = True,
    # metric_for_best_model = 'accuracy',
    # run_name = 'custom_training'            # name of the W&B run

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,            # for padding batched data
    compute_metrics=compute_metrics
)

PyTorch: setting up devices


In [80]:
from timeit import default_timer as timer
start = timer()

trainer.train()


end = timer()
print("%4f seconds, %4f minutes elapsed" % (float(end-start), float((end-start)/60)))

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14067
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1759
  Number of trainable parameters = 109483778
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
500,0.370800
1000,0.307900
1500,0.279600


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tmp/tmp7qc0kwc5
Configuration saved in /tmp/tmp7qc0kwc5/config.json
Model weights saved in /tmp/tmp7qc0kwc5/pytorch_model.bin


1308.139700 seconds, 21.802328 minutes elapsed


In [81]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 3517
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.2499202936887741,
 'eval_accuracy': 0.9033266988911004,
 'eval_precision': 0.7768361581920904,
 'eval_recall': 0.7513661202185792,
 'eval_f1': 0.763888888888889,
 'eval_runtime': 112.6355,
 'eval_samples_per_second': 31.225,
 'eval_steps_per_second': 3.906,
 'epoch': 1.0}

In [82]:
np.set_printoptions(suppress=True)


In [105]:
text = "Super charging is working very well."
# text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)    # also gelu(), silu()
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.8727, -1.0109]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[ 0.6155, -0.2697]], device='cuda:0', grad_fn=<SiluBackward0>)


array([[ 0.61548567, -0.26971212]], dtype=float32)

In [84]:
trainer.save_model('BertPractice')

Saving model checkpoint to BertPractice
Configuration saved in BertPractice/config.json
Model weights saved in BertPractice/pytorch_model.bin


In [85]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [86]:
model_2 = BertForSequenceClassification.from_pretrained('BertPractice')
model_2.to('cuda')

loading configuration file BertPractice/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file BertPractice/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [115]:
# text = "Super charging is working very well."
text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.09049978, 0.90950024]], dtype=float32)

In [140]:

#today's VOC
# import file and combine title & content
voctoday = pd.read_csv('/0119.csv', sep='\t', index_col=None).drop_duplicates()
voctoday['title'] = voctoday['title'].astype(str) + " " + voctoday['content'].astype(str)
voctoday = voctoday[['title', 'class']]

voclist = list(voctoday['title'])
predlist = []

for i in range(len(voclist)):
  inputs = tokenizer(voclist[i], padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs = model_2(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  predlist.append(predictions)

organizedzip = zip([x[:25] for x in voclist],
                   predlist,
                   voctoday['class'])
    
# results as voc text, predictions, and actual value
print(list(organizedzip))


[('2nd January update I just', array([[0.9971807 , 0.00281931]], dtype=float32), 'N'), ('No Questions just wanted ', array([[0.9981193 , 0.00188064]], dtype=float32), 'N'), ('S22u vs S23u Having time ', array([[0.9957015 , 0.00429845]], dtype=float32), 'N'), ('No more lag and stutter a', array([[0.7561109, 0.2438891]], dtype=float32), 'N'), ('Qualcomm Bluetooth Codecs', array([[0.99762386, 0.00237617]], dtype=float32), 'N'), ('Sound Issues Hey everyone', array([[0.07707566, 0.92292434]], dtype=float32), 'R'), ('Samsung Leather Case vs O', array([[0.9981981, 0.0018019]], dtype=float32), 'N'), ('Weather widget help nan', array([[0.97075576, 0.02924431]], dtype=float32), 'R'), ('battery concern I can squ', array([[0.9247212 , 0.07527879]], dtype=float32), 'R'), ('S22 Ultra Camera Issued H', array([[0.06688055, 0.9331195 ]], dtype=float32), 'R'), ('Single Take Functionality', array([[0.97552943, 0.02447058]], dtype=float32), 'N'), ('So I already had the Janu', array([[0.9965773 , 0.0034226

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5caba9f-cd36-4d50-aaa3-2cf59957a2f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>